In [1]:
import pandas as pd

In [49]:
predictions = pd.read_csv("submission.csv")
seeds = pd.read_csv("kaggle/input/march-machine-learning-mania-2023/MNCAATourneySeeds.csv")
teams = pd.read_csv("kaggle/input/march-machine-learning-mania-2023/MTeams.csv")

predictions[["Season", "TeamIDLow", "TeamIDHigh"]] = predictions["ID"].str.split("_", expand=True).astype(int)
predictions["ProbIDLowWin"] = predictions["Pred"]
predictions.drop(["ID", "Pred", "Season"], axis=1, inplace=True)

seeds = seeds[seeds["Season"] == 2023]
seeds.drop("Season", axis=1, inplace=True)

teams.drop(["FirstD1Season", "LastD1Season"], axis=1, inplace=True)

seeds = seeds.merge(teams, how="left", on="TeamID")

seeds_low = seeds.copy()
seeds_low.columns = [f"{name}Low" for name in seeds_low.columns]
seeds_high = seeds.copy()
seeds_high.columns = [f"{name}High" for name in seeds_high.columns]

predictions = predictions.merge(seeds_low, how="inner", on="TeamIDLow")
predictions = predictions.merge(seeds_high, how="inner", on="TeamIDHigh")

predictions.head()

,TeamIDLow,TeamIDHigh,ProbIDLowWin,SeedLow,TeamNameLow,SeedHigh,TeamNameHigh
0,1104,1112,0.638275,X01,Alabama,X02,Arizona
1,1104,1113,0.500000,X01,Alabama,Z11a,Arizona St
2,1112,1113,0.500000,X02,Arizona,Z11a,Arizona St
3,1104,1116,0.813497,X01,Alabama,Z08,Arkansas
4,1112,1116,0.711978,X02,Arizona,Z08,Arkansas


In [76]:
first_round = predictions[
    (predictions["SeedLow"].str[0] == predictions["SeedHigh"].str[0]) & 
    (predictions["SeedLow"].str[1:3].astype(int) == (17 - predictions["SeedHigh"].str[1:3].astype(int)))
].sort_values(["SeedLow", "TeamIDHigh"])
first_round.loc[first_round["ProbIDLowWin"] < 0.5, "Pick"] = first_round["TeamNameHigh"]
first_round.loc[first_round["ProbIDLowWin"] > 0.5, "Pick"] = first_round["TeamNameLow"]
first_round.loc[(first_round["ProbIDLowWin"] == 0.5) & (first_round["SeedLow"].str[1:3].astype(int) < first_round["SeedHigh"].str[1:3].astype(int)), "Pick"] = first_round["TeamNameLow"]
first_round.loc[(first_round["ProbIDLowWin"] == 0.5) & (first_round["SeedLow"].str[1:3].astype(int) > first_round["SeedHigh"].str[1:3].astype(int)), "Pick"] = first_round["TeamNameHigh"]

first_round

,TeamIDLow,TeamIDHigh,ProbIDLowWin,SeedLow,TeamNameLow,SeedHigh,TeamNameHigh,Pick
1643,1345,1411,0.977989,W01,Purdue,W16b,TX Southern,Purdue
2046,1266,1436,0.919188,W02,Marquette,W15,Vermont,Marquette
692,1243,1286,0.500000,W03,Kansas St,W14,Montana St,Kansas St
1765,1397,1418,0.500000,W04,Tennessee,W13,Louisiana,Tennessee
873,1181,1331,0.500000,W05,Duke,W12,Oral Roberts,Duke
1064,1246,1344,0.870311,W06,Kentucky,W11,Providence,Kentucky
1864,1277,1425,0.681650,W07,Michigan St,W10,USC,Michigan St
510,1194,1272,0.500000,W09,FL Atlantic,W08,Memphis,Memphis
1094,1192,1345,0.500000,W16a,F Dickinson,W01,Purdue,Purdue
1225,1104,1369,0.500000,X01,Alabama,X16a,SE Missouri St,Alabama


In [115]:
ids = sorted([1222 , 1345])
predictions[(predictions["TeamIDLow"] == ids[0]) & (predictions["TeamIDHigh"] == ids[1])]

,TeamIDLow,TeamIDHigh,ProbIDLowWin,SeedLow,TeamNameLow,SeedHigh,TeamNameHigh
1099,1222,1345,0.63818,Y01,Houston,W01,Purdue
